This notebook is created to retrieve and parse the functional information associated to 6 organism models about their protein sequences. The final goal is to create a format available to visualize in Tableau (e.g tsv format) and explore them at the functional level of the orthology relationships among inferred transcriptional regulatory interactions.

We'll create (see below) three folders to save our results: *InterproResults*, containing the output of interproscan; *InterproParsedXML*, containing the outputs we'll create and; *InterproFullMerge*, the final tables with all the info

In [1]:
%%bash

# If folders dont exist, create them

if [ ! -d InterproResults ]; then
    mkdir InterproResults
fi

if [ ! -d InterproParsedXML ]; then 
    mkdir InterproParsedXML
fi

if [ ! -d InterproFullMerge ]; then
    mkdir InterproFullMerge
fi

## Preprocessing

In order to retrieve information asscociated about the function of proteins/domains of the sequences, interproscan (only available for based-Linux systems) was used for the six organisms. Basically, Interproscan was ran as is shown below:

In [2]:
%%bash

set -e

cd InterproResults
for fasta in ../genomes/*faa; do

        # Uncomment the following line to run interproscan and replace the path with your
        # current sets of interproscan
        #./../../../../Descargas/Interpro/interproscan-5.60-92.0/interproscan.sh --verbose -i "${fasta}" -f xml,tsv -appl PANTHER,Pfam,FunFam,SUPERFAMILY,CDD --goterms
        # To give the opportunity to release memory before the following fasta
        sleep 5
done

cd ..

Five methods were used (PANTHER,Pfam,FunFam,SUPERFAMILY,CDD) and two output formats were asked. A tsv file containing the structure we are interested to and, a xml containing a wealthier description about the matches of the sequences. Each file is named as the fasta file plus the respective extention, for instance, if the file of the protein sequences of _E.coli_ is **GCF_000005845.2_E_coli_K12_genomic.faa**, the interproscan output will be **GCF_000005845.2_E_coli_K12_genomic.faa.tsv** and **GCF_000005845.2_E_coli_K12_genomic.faa.xml**

We will cross the information of both files for each genome. An overview of both files are shown in the following cells

In [3]:
import pandas as pd
import numpy as np
import os

In [4]:
path_interpro_results = "InterproResults"
all_files_names_results = os.listdir(path_interpro_results)

# Filtering only the files ending with "faa.tsv"
tsv_files_names_results = [f for f in all_files_names_results if f.endswith("faa.tsv") ]

In [5]:
tsv_files_names_results.sort()
tsv_files_names_results

['GCF_000005845.2_E_coli_K12_genomic.faa.tsv',
 'GCF_000006765.1_ASM676v1_P_aeruginosa_PA01_genomic.faa.tsv',
 'GCF_000006945.2_ASM694v2_S_enterica_LT2_genomic.faa.tsv',
 'GCF_000009045.1_ASM904v1_B_subtilis_168_genomic.faa.tsv',
 'GCF_000009645.1_ASM964v1_S_aureus_N315_genomic.faa.tsv',
 'GCF_000195955.2_ASM19595v2_M_tuberculosis_H37Rv_genomic.faa.tsv']

In [6]:
# We'll load the files later, here I just show one of them
pd.read_csv(os.path.join(path_interpro_results, "GCF_000005845.2_E_coli_K12_genomic.faa.tsv"),
            sep="\t",
            header=None,
            index_col=None,
            nrows=5)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,NP_416254.1,ea37cb76b651f90db9b8d4dbf6861145,275,CDD,cd00553,NAD_synthase,17,262,1.857550e-84,T,08-02-2023,IPR003694,NAD(+) synthetase,GO:0003952|GO:0004359|GO:0005737|GO:0009435
1,NP_416254.1,ea37cb76b651f90db9b8d4dbf6861145,275,PANTHER,PTHR23090,NH 3 /GLUTAMINE-DEPENDENT NAD + SYNTHETASE,11,242,7.600000e-32,T,08-02-2023,IPR003694,NAD(+) synthetase,GO:0003952|GO:0004359|GO:0005737|GO:0009435
2,NP_416254.1,ea37cb76b651f90db9b8d4dbf6861145,275,FunFam,G3DSA:3.40.50.620:FF:000015,NH(3)-dependent NAD(+) synthetase,1,274,0.000000e+00,T,08-02-2023,-,-,NaN
3,NP_416254.1,ea37cb76b651f90db9b8d4dbf6861145,275,SUPERFAMILY,SSF52402,Adenine nucleotide alpha hydrolases-like,3,273,1.450000e-86,T,08-02-2023,-,-,NaN
4,NP_416254.1,ea37cb76b651f90db9b8d4dbf6861145,275,Pfam,PF02540,NAD synthase,23,265,1.600000e-81,T,08-02-2023,IPR022310,NAD/GMP synthase,-


In [7]:
%%bash

head -n 20 "InterproResults/GCF_000005845.2_E_coli_K12_genomic.faa.xml"

<?xml version="1.0" encoding="UTF-8"?><protein-matches xmlns="http://www.ebi.ac.uk/interpro/resources/schemas/interproscan5" interproscan-version="5.60-92.0">
  <protein>
    <sequence md5="ea37cb76b651f90db9b8d4dbf6861145">MTLQQQIIKALGAKPQINAEEEIRRSVDFLKSYLQTYPFIKSLVLGISGGQDSTLAGKLCQMAINELRLETGNESLQFIAVRLPYGVQADEQDCQDAIAFIQPDRVLTVNIKGAVLASEQALREAGIELSDFVRGNEKARERMKAQYSIAGMTSGVVVGTDHAAEAITGFFTKYGDGGTDINPLYRLNKRQGKQLLAALACPEHLYKKAPTADLEDDRPSLPDEVALGVTYDNIDDYLEGKNVPQQVARTIENWYLKTEHKRRPPITVFDDFWKK</sequence>
    <xref id="NP_416254.1" name="NP_416254.1&#9;nadE|NAD synthetase, NH3-dependent|Escherichia coli str. K-12 substr. MG1655"/>
    <matches>
      <hmmer3-match evalue="0.0" score="522.3">
        <signature ac="G3DSA:3.40.50.620:FF:000015" desc="NH(3)-dependent NAD(+) synthetase">
          <signature-library-release library="FUNFAM" version="4.3.0"/>
        </signature>
        <model-ac>3.40.50.620-FF-000015</model-ac>
        <locations>
          <hmmer3-location env-end="274" 

We need to transform the XMF file into something more easy to manipulate and cross with the tsv file. As an initial approach, I'll use RegEx to filter the tags of **xreg**;containing the sequence ID, **signature ac**;the id of the method and job used;**entry ac**; general information and **go-xref category**; descriptions of GO terms

First, using the GO terms we'll construct a reference table using bash about all descriptions of the proteins and we'll keep that table separated. In theory, each GO term has a unique ID and that is the reason why this should work.

In addition, we'll add a value "NaN" to group all the terms with no information and also the header.

In [3]:
%%bash

# Empty file
printf "" > GO_terms_tmp.tsv

for XML in InterproResults/*xml; do

    # Retrieve go terms; remove attribute names; remove leading spaces and 
    # replace the first two blank spaces by tabs
    PRE_CLEANED_GO=$(grep -E "<go-xref\s+category" "${XML}" |
                         sed -r 's/(<go-xref category=|db="GO"\s+id=|name=|\/>|")//g' |
                         sed -r 's/^\s+//g' |
                         sed -r 's/\s+/\t/1' |
                         sed -r 's/\s+/\t/2')
    
    echo "${PRE_CLEANED_GO}" >> GO_terms_tmp.tsv

done

# Removing go terms repeated
sort GO_terms_tmp.tsv | uniq > GO_terms.tsv && rm GO_terms_tmp.tsv

# Adding NaN at the end and headers at the beginning
printf "NaN\tNaN\tNaN\n" >> GO_terms.tsv
sed -i "1s/^/category\tid_go\tdescription_go\n/" GO_terms.tsv

column -s$'\t' -t GO_terms.tsv | head

category            id_go       description_go
BIOLOGICAL_PROCESS  GO:0000041  transition metal ion transport
BIOLOGICAL_PROCESS  GO:0000103  sulfate assimilation
BIOLOGICAL_PROCESS  GO:0000105  histidine biosynthetic process
BIOLOGICAL_PROCESS  GO:0000160  phosphorelay signal transduction system
BIOLOGICAL_PROCESS  GO:0000162  tryptophan biosynthetic process
BIOLOGICAL_PROCESS  GO:0000256  allantoin catabolic process
BIOLOGICAL_PROCESS  GO:0000271  polysaccharide biosynthetic process
BIOLOGICAL_PROCESS  GO:0000272  polysaccharide catabolic process
BIOLOGICAL_PROCESS  GO:0000413  protein peptidyl-prolyl isomerization


As you can see, the new file named *GO_terms.tsv* has three columns: first one with the category of the GO term, a second one with the GO id and the last one with the description about that term.

Now it's time to process the remained labels. Because of the complexity of the XML files (e.g many nested tags) the use of libraries such as ```xml.etree.ElementTree``` in pyhon does not seem appropiate in terms of for loops needed and maybe in terms of legibility (but I'll use regex haha), same case for trying to use ```pandas``` so again, I'll do that using bash, feel free to find another way to acomplish this

In [4]:
%%bash

for XML in InterproResults/*xml; do

    OUTPUT_NAME=$(basename "${XML}" | sed -r 's/$/\.tsv/g')

    # Retrieve all tags and attributes; retrieve only important attributes; clean attribute names 
    PRE_CLEANED_XML=$(grep -P "(<xref |<signature |<entry ac)" "${XML}" |
                          grep -Po '(<xref id="[^"]+"|<signature ac="[^"]+"|type="[^"]+")' |
                          sed -r 's/^<\w+ //g')

    # Remove quotes ", merge type with method ID into a single row, clean attributes names
    PRE_FORMATED_XML=$(echo "${PRE_CLEANED_XML}" |
                           sed -r 's/"//g' |
                           sed ':r;$!{N;br};s/\ntype/\ttype/g' |
                           sed -r 's/ac=|type=//g')

    # Parser protein ID (clear format) and get tabular form
    echo "${PRE_FORMATED_XML}" |
        perl -ne 'if($_ =~ /id=/){
                        chomp($_);
                        $header= ($_ =~ s/id=//r);
                } else{
                        chomp($_);
                        $new_string=$header . "\t" . $_;
                        $len = 3 - scalar(split("\t", $new_string));
                        print $new_string,"\tNULL_TYPE"x$len,"\n"
                }' > InterproParsedXML/$OUTPUT_NAME

done

# Ecoli file
column -s$'\t' -t InterproParsedXML/GCF_000005845.2_E_coli_K12_genomic.faa.xml.tsv | head -n 10

NP_416254.1     G3DSA:3.40.50.620:FF:000015    NULL_TYPE
NP_416254.1     PF02540                        DOMAIN
NP_416254.1     PTHR23090                      FAMILY
NP_416254.1     cd00553                        FAMILY
NP_416254.1     SSF52402                       NULL_TYPE
NP_416630.1     PF07694                        DOMAIN
NP_416630.1     PF06580                        DOMAIN
NP_416630.1     G3DSA:3.30.450.40:FF:000013    NULL_TYPE
NP_416630.1     PTHR34220                      NULL_TYPE
NP_416630.1     cd16956                        NULL_TYPE


From this file, we can load and map by protein ID and Method ID the XML file parsed and the tsv from interproscan for every organism, we'll take _E.coli_ as an example of the logic to follow:

In [10]:
# Reading two into two dataFrames the tsv file from interproscan and the tsv file obtained from the 
# processing of the xml file

path_interpro_parsed = "InterproParsedXML/"

xml_parsed_df = pd.read_csv(os.path.join(path_interpro_parsed,"GCF_000005845.2_E_coli_K12_genomic.faa.xml.tsv"),
                            header=None,
                            sep="\t")

tsv_df = pd.read_csv(os.path.join(path_interpro_results, "GCF_000005845.2_E_coli_K12_genomic.faa.tsv"),
                     header=None,
                     sep="\t")

In [11]:
# Creating an index based on protein ID (column 0) and method ID (column 4).
# This index will be shared between the two dataframes to cross the information later

tsv_df["mul_index"] = tsv_df[0] + "\t" + tsv_df[4]

# Replacing NULL values (score -) by np.NaN
for col in tsv_df:
    tsv_df[col] = tsv_df[col].replace("-", np.NaN)

# Replacing the NULL values ("NULL_TYPE") by np.NaN
xml_parsed_df[2] = xml_parsed_df[2].replace("NULL_TYPE", np.NaN)
xml_parsed_df["mul_index"] = xml_parsed_df[0] + "\t" + xml_parsed_df[1]

In [12]:
xml_parsed_df.head(3)

,0,1,2,mul_index
0,NP_416254.1,G3DSA:3.40.50.620:FF:000015,NaN,NP_416254.1\tG3DSA:3.40.50.620:FF:000015
1,NP_416254.1,PF02540,DOMAIN,NP_416254.1\tPF02540
2,NP_416254.1,PTHR23090,FAMILY,NP_416254.1\tPTHR23090


In [13]:
tsv_df.head(3)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,mul_index
0,NP_416254.1,ea37cb76b651f90db9b8d4dbf6861145,275,CDD,cd00553,NAD_synthase,17,262,1.857550e-84,T,08-02-2023,IPR003694,NAD(+) synthetase,GO:0003952|GO:0004359|GO:0005737|GO:0009435,NP_416254.1\tcd00553
1,NP_416254.1,ea37cb76b651f90db9b8d4dbf6861145,275,PANTHER,PTHR23090,NH 3 /GLUTAMINE-DEPENDENT NAD + SYNTHETASE,11,242,7.600000e-32,T,08-02-2023,IPR003694,NAD(+) synthetase,GO:0003952|GO:0004359|GO:0005737|GO:0009435,NP_416254.1\tPTHR23090
2,NP_416254.1,ea37cb76b651f90db9b8d4dbf6861145,275,FunFam,G3DSA:3.40.50.620:FF:000015,NH(3)-dependent NAD(+) synthetase,1,274,0.000000e+00,T,08-02-2023,NaN,NaN,NaN,NP_416254.1\tG3DSA:3.40.50.620:FF:000015


In [14]:
# Left Join by mul_index
full_tsv = tsv_df.merge(xml_parsed_df, on="mul_index", how="left")
full_tsv.drop(["mul_index","0_y","1_y"], axis=1, inplace=True)
full_tsv.head(3)

,0_x,1_x,2_x,3,4,5,6,7,8,9,10,11,12,13,2_y
0,NP_416254.1,ea37cb76b651f90db9b8d4dbf6861145,275,CDD,cd00553,NAD_synthase,17,262,1.857550e-84,T,08-02-2023,IPR003694,NAD(+) synthetase,GO:0003952|GO:0004359|GO:0005737|GO:0009435,FAMILY
1,NP_416254.1,ea37cb76b651f90db9b8d4dbf6861145,275,PANTHER,PTHR23090,NH 3 /GLUTAMINE-DEPENDENT NAD + SYNTHETASE,11,242,7.600000e-32,T,08-02-2023,IPR003694,NAD(+) synthetase,GO:0003952|GO:0004359|GO:0005737|GO:0009435,FAMILY
2,NP_416254.1,ea37cb76b651f90db9b8d4dbf6861145,275,FunFam,G3DSA:3.40.50.620:FF:000015,NH(3)-dependent NAD(+) synthetase,1,274,0.000000e+00,T,08-02-2023,NaN,NaN,NaN,NaN


In the *GO* column we have multiples values in a single row, in order to connect the table we are creating with the **GO_terms.tsv** table, we'll convert the DataFrame in a "long" format dataframe expanding the multiples values found in *GO* to multiples rows describing the same result but, this time, each row will have only one GO term as exemplified below:

In [15]:
ids = np.arange(0,3,1)
go_terms = ["GO:1|GO:2",np.NaN,"GO:3"]

df_shown = pd.DataFrame({"ids":ids, "go":go_terms})
df_shown

,ids,go
0,0,GO:1|GO:2
1,1,NaN
2,2,GO:3


In [16]:
ids = np.array([0,0,1,2])
go_terms = ["GO:1","GO:2", np.NaN, "GO:3"]

df_expected = pd.DataFrame({"ids":ids, "go":go_terms})
df_expected

,ids,go
0,0,GO:1
1,0,GO:2
2,1,NaN
3,2,GO:3


Note that maintaining the index is important since it will be used to map between this new format and the interproscan table format we want in the end.

In [17]:
# Convert the GO terms found into a dataframe of multiples columns depending of the number of GO terms
full_go_terms_df = full_tsv[13].str.split("|").apply(pd.Series)

# "Merge" all the columns from the same index into multiples rows of one colum
full_go_terms_merged_df = full_go_terms_df.copy().T
full_go_terms_merged_df = full_go_terms_merged_df.melt(var_name="index_tmp").drop_duplicates()

# Masking to remove duplicated index with np.NaN value
mask = ~(full_go_terms_merged_df["index_tmp"].duplicated(keep=False) & full_go_terms_merged_df["value"].isna())
full_go_terms_merged_df = full_go_terms_merged_df[mask].copy().set_index("index_tmp")

# Left join with the interproscan table we have been processing in the last cells
full_df = full_tsv.merge(full_go_terms_merged_df, how="left", right_index=True, left_index=True).copy()
full_df.head()

,0_x,1_x,2_x,3,4,5,6,7,8,9,10,11,12,13,2_y,value
0,NP_416254.1,ea37cb76b651f90db9b8d4dbf6861145,275,CDD,cd00553,NAD_synthase,17,262,1.857550e-84,T,08-02-2023,IPR003694,NAD(+) synthetase,GO:0003952|GO:0004359|GO:0005737|GO:0009435,FAMILY,GO:0003952
0,NP_416254.1,ea37cb76b651f90db9b8d4dbf6861145,275,CDD,cd00553,NAD_synthase,17,262,1.857550e-84,T,08-02-2023,IPR003694,NAD(+) synthetase,GO:0003952|GO:0004359|GO:0005737|GO:0009435,FAMILY,GO:0004359
0,NP_416254.1,ea37cb76b651f90db9b8d4dbf6861145,275,CDD,cd00553,NAD_synthase,17,262,1.857550e-84,T,08-02-2023,IPR003694,NAD(+) synthetase,GO:0003952|GO:0004359|GO:0005737|GO:0009435,FAMILY,GO:0005737
0,NP_416254.1,ea37cb76b651f90db9b8d4dbf6861145,275,CDD,cd00553,NAD_synthase,17,262,1.857550e-84,T,08-02-2023,IPR003694,NAD(+) synthetase,GO:0003952|GO:0004359|GO:0005737|GO:0009435,FAMILY,GO:0009435
1,NP_416254.1,ea37cb76b651f90db9b8d4dbf6861145,275,PANTHER,PTHR23090,NH 3 /GLUTAMINE-DEPENDENT NAD + SYNTHETASE,11,242,7.600000e-32,T,08-02-2023,IPR003694,NAD(+) synthetase,GO:0003952|GO:0004359|GO:0005737|GO:0009435,FAMILY,GO:0003952


In [18]:
full_df

,0_x,1_x,2_x,3,4,5,6,7,8,9,10,11,12,13,2_y,value
0,NP_416254.1,ea37cb76b651f90db9b8d4dbf6861145,275,CDD,cd00553,NAD_synthase,17,262,1.857550e-84,T,08-02-2023,IPR003694,NAD(+) synthetase,GO:0003952|GO:0004359|GO:0005737|GO:0009435,FAMILY,GO:0003952
0,NP_416254.1,ea37cb76b651f90db9b8d4dbf6861145,275,CDD,cd00553,NAD_synthase,17,262,1.857550e-84,T,08-02-2023,IPR003694,NAD(+) synthetase,GO:0003952|GO:0004359|GO:0005737|GO:0009435,FAMILY,GO:0004359
0,NP_416254.1,ea37cb76b651f90db9b8d4dbf6861145,275,CDD,cd00553,NAD_synthase,17,262,1.857550e-84,T,08-02-2023,IPR003694,NAD(+) synthetase,GO:0003952|GO:0004359|GO:0005737|GO:0009435,FAMILY,GO:0005737
0,NP_416254.1,ea37cb76b651f90db9b8d4dbf6861145,275,CDD,cd00553,NAD_synthase,17,262,1.857550e-84,T,08-02-2023,IPR003694,NAD(+) synthetase,GO:0003952|GO:0004359|GO:0005737|GO:0009435,FAMILY,GO:0009435
1,NP_416254.1,ea37cb76b651f90db9b8d4dbf6861145,275,PANTHER,PTHR23090,NH 3 /GLUTAMINE-DEPENDENT NAD + SYNTHETASE,11,242,7.600000e-32,T,08-02-2023,IPR003694,NAD(+) synthetase,GO:0003952|GO:0004359|GO:0005737|GO:0009435,FAMILY,GO:0003952
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21627,NP_416130.3,db79e4dbecd21e0c3a7736850a23e810,391,FunFam,G3DSA:2.60.120.10:FF:000076,Mannose-6-phosphate isomerase,301,382,1.400000e-53,T,08-02-2023,NaN,NaN,NaN,NaN,NaN
21628,NP_416130.3,db79e4dbecd21e0c3a7736850a23e810,391,Pfam,PF01238,Phosphomannose isomerase type I C-terminal,310,357,9.800000e-16,T,08-02-2023,IPR046456,"Phosphomannose isomerase type I, C-terminal do...",NaN,DOMAIN,NaN
21629,NP_416130.3,db79e4dbecd21e0c3a7736850a23e810,391,Pfam,PF20511,"Phosphomannose isomerase type I, catalytic domain",1,150,2.300000e-54,T,08-02-2023,IPR046457,"Phosphomannose isomerase type I, catalytic domain",GO:0004476|GO:0008270,DOMAIN,GO:0004476
21629,NP_416130.3,db79e4dbecd21e0c3a7736850a23e810,391,Pfam,PF20511,"Phosphomannose isomerase type I, catalytic domain",1,150,2.300000e-54,T,08-02-2023,IPR046457,"Phosphomannose isomerase type I, catalytic domain",GO:0004476|GO:0008270,DOMAIN,GO:0008270


In [19]:
# Now lets remove the "GO" column and rename the columns according to the doc of interproscan

full_df.drop(13, axis=1, inplace=True)

full_df.rename({"0_x": "id",
                 "1_x":"md5",
                 "2_x": "protein_len",
                 3: "method", 
                 4:"id_method", 
                 5:"description", 
                 6:"start_loc", 
                 7:"end_loc", 
                 8:"score", 
                 9:"status", 
                 10:"date", 
                 11:"interpro_access", 
                 12:"interpro_description", 
                 "2_y":"type", 
                 "value":"GO"}, axis=1, inplace=True)

full_df.head(5)

,id,md5,protein_len,method,id_method,description,start_loc,end_loc,score,status,date,interpro_access,interpro_description,type,GO
0,NP_416254.1,ea37cb76b651f90db9b8d4dbf6861145,275,CDD,cd00553,NAD_synthase,17,262,1.857550e-84,T,08-02-2023,IPR003694,NAD(+) synthetase,FAMILY,GO:0003952
0,NP_416254.1,ea37cb76b651f90db9b8d4dbf6861145,275,CDD,cd00553,NAD_synthase,17,262,1.857550e-84,T,08-02-2023,IPR003694,NAD(+) synthetase,FAMILY,GO:0004359
0,NP_416254.1,ea37cb76b651f90db9b8d4dbf6861145,275,CDD,cd00553,NAD_synthase,17,262,1.857550e-84,T,08-02-2023,IPR003694,NAD(+) synthetase,FAMILY,GO:0005737
0,NP_416254.1,ea37cb76b651f90db9b8d4dbf6861145,275,CDD,cd00553,NAD_synthase,17,262,1.857550e-84,T,08-02-2023,IPR003694,NAD(+) synthetase,FAMILY,GO:0009435
1,NP_416254.1,ea37cb76b651f90db9b8d4dbf6861145,275,PANTHER,PTHR23090,NH 3 /GLUTAMINE-DEPENDENT NAD + SYNTHETASE,11,242,7.600000e-32,T,08-02-2023,IPR003694,NAD(+) synthetase,FAMILY,GO:0003952


Now we need to do the same for every organism, I'll create some functions of the codes shown above, so they are going to do all the hard work. In addition, a short code will be added to save into a file the tables created

In [20]:
def LoadFiles(xml_file_name:str, tsv_file_name:str, keydict:dict=None) -> (pd.DataFrame, pd.DataFrame):
    
    # This function read a tsv file and a parsed xml file obtained from interproscan into a pandas DataFrame
    
    print(f"Reading files into DataFrames: {xml_file_name}")
    print(f"                               {tsv_file_name}")
    
    xml_df = pd.read_csv(xml_file_name, **keydict)
    tsv_df = pd.read_csv(tsv_file_name, **keydict)
    
    return xml_df, tsv_df
    
def ParseFrames(xml_df:pd.DataFrame, tsv_df:pd.DataFrame, shared_index:str) -> (pd.DataFrame, pd.DataFrame):
    
    # This function create a shared index between two data frames and replace null values by np.NaN
    
    print(f"Parsing frames using the shared_index = {shared_index}")
    
    tsv_df[shared_index] = tsv_df[0] + "\t" + tsv_df[4]

    # Replacing NULL values (score -) by NaN
    for col in tsv_df:
        tsv_df[col] = tsv_df[col].replace("-", np.NaN)

    xml_df[2] = xml_df[2].replace("NULL_TYPE", np.NaN)
    xml_df[shared_index] = xml_df[0] + "\t" + xml_df[1]
    
    return xml_df, tsv_df

def JoinFrames(left:pd.DataFrame, right:pd.DataFrame, keydict:dict, drop:list=None) -> pd.DataFrame:
    
    # This function do a join operation between two dataframes
    
    print(f"Joining frames with parameters: {keydict}")
    df = left.merge(right, **keydict)
    
    if drop:
        df.drop(drop, axis=1, inplace=True)
    
    return df

def ParseGo(data:pd.DataFrame, col_target, sep_target, keydict:dict) -> pd.DataFrame:
    
    # This function merge the GO term found in a single-line format in a dataframe 
    # into multiples lines containing only one GO term in a dataframe  
    
    print(f"Parsing GO terms with parameters {keydict}")
    
    data_go = data[col_target].str.split(sep_target).apply(pd.Series)

    # "Merge" all the columns from the same index into multiples rows of one colum
    data_go_merged = data_go.copy().T
    data_go_merged = data_go_merged.melt(var_name="index_tmp").drop_duplicates()

    # Masking to remove duplicated index with np.NaN value
    mask = ~(data_go_merged["index_tmp"].duplicated(keep=False) & data_go_merged["value"].isna())
    data_go_merged = data_go_merged[mask].copy().set_index("index_tmp")

    # Left join with the interproscan table we have been processing in the last cells
    full_df = data.merge(data_go_merged, **keydict).copy()
    
    return full_df.drop(col_target, axis=1)
    
def RenameFrame(DataFrame:pd.DataFrame) -> pd.DataFrame:
    
    # This function rename the columns of a dataframe according to the interpro documentation
    print("Renaming frame")
    
    DataFrame.rename({"0_x": "id",
                 "1_x":"md5",
                 "2_x": "protein_len",
                 3: "method", 
                 4:"id_method", 
                 5:"description", 
                 6:"start_loc", 
                 7:"end_loc", 
                 8:"score", 
                 9:"status", 
                 10:"date", 
                 11:"interpro_access", 
                 12:"interpro_description", 
                 "2_y":"type", 
                 "value":"GO"}, axis=1, inplace=True)
    
    return DataFrame

TSV file are already loaded, we'll need to load now the new created xml.tsv files

In [21]:
all_files_names_parsed = os.listdir(path_interpro_parsed)

# Filter only the files ending with "xml.tsv"
xml_tsv_files_names = [f for f in all_files_names_parsed if f.endswith("xml.tsv") ]

In [22]:
xml_tsv_files_names.sort()
xml_tsv_files_names

['GCF_000005845.2_E_coli_K12_genomic.faa.xml.tsv',
 'GCF_000006765.1_ASM676v1_P_aeruginosa_PA01_genomic.faa.xml.tsv',
 'GCF_000006945.2_ASM694v2_S_enterica_LT2_genomic.faa.xml.tsv',
 'GCF_000009045.1_ASM904v1_B_subtilis_168_genomic.faa.xml.tsv',
 'GCF_000009645.1_ASM964v1_S_aureus_N315_genomic.faa.xml.tsv',
 'GCF_000195955.2_ASM19595v2_M_tuberculosis_H37Rv_genomic.faa.xml.tsv']

In [23]:
tsv_files_names_results

['GCF_000005845.2_E_coli_K12_genomic.faa.tsv',
 'GCF_000006765.1_ASM676v1_P_aeruginosa_PA01_genomic.faa.tsv',
 'GCF_000006945.2_ASM694v2_S_enterica_LT2_genomic.faa.tsv',
 'GCF_000009045.1_ASM904v1_B_subtilis_168_genomic.faa.tsv',
 'GCF_000009645.1_ASM964v1_S_aureus_N315_genomic.faa.tsv',
 'GCF_000195955.2_ASM19595v2_M_tuberculosis_H37Rv_genomic.faa.tsv']

In [24]:
# Arguments for the functions and folder to save files
output_dir = "InterproFullMerge"
arguments_load_function = {"header": None, "sep":"\t"}
arguments_join_function = {"on":"mul_index", "how":"left"}
arguments_parse_go_function = {"how":"left", "right_index":True, "left_index":True}

# Doing the same as shown above but using all organisms
for t_f, x_f in zip(tsv_files_names_results, xml_tsv_files_names):
    
    tsv_path = os.path.join(path_interpro_results, t_f)
    xml_path = os.path.join(path_interpro_parsed, x_f)
    
    xml_df, tsv_df = LoadFiles(xml_path, tsv_path, arguments_load_function)
    xml_df, tsv_df = ParseFrames(xml_df, tsv_df, "mul_index")
    
    full_df = JoinFrames(tsv_df, xml_df, arguments_join_function, ["mul_index","0_y","1_y"])
    full_df = ParseGo(full_df, 13, "|", arguments_parse_go_function)
    
    full_df = RenameFrame(full_df)
    
    # Saving frame into a file ending with ".full" 
    output_file_path = os.path.join(output_dir, t_f + ".full.tsv")
    print(f"Saving frame into {output_file_path}\n")
    full_df.to_csv(output_file_path, sep="\t", header=True, index=True, na_rep="NaN", index_label="id_row")

print("All done")

Reading files into DataFrames: InterproParsedXML/GCF_000005845.2_E_coli_K12_genomic.faa.xml.tsv
                               InterproResults/GCF_000005845.2_E_coli_K12_genomic.faa.tsv
Parsing frames using the shared_index = mul_index
Joining frames with parameters: {'on': 'mul_index', 'how': 'left'}
Parsing GO terms with parameters {'how': 'left', 'right_index': True, 'left_index': True}
Renaming frame
Saving frame into InterproFullMerge/GCF_000005845.2_E_coli_K12_genomic.faa.tsv.full.tsv

Reading files into DataFrames: InterproParsedXML/GCF_000006765.1_ASM676v1_P_aeruginosa_PA01_genomic.faa.xml.tsv
                               InterproResults/GCF_000006765.1_ASM676v1_P_aeruginosa_PA01_genomic.faa.tsv
Parsing frames using the shared_index = mul_index
Joining frames with parameters: {'on': 'mul_index', 'how': 'left'}
Parsing GO terms with parameters {'how': 'left', 'right_index': True, 'left_index': True}
Renaming frame
Saving frame into InterproFullMerge/GCF_000006765.1_ASM676v1_P_